In [170]:
import pandas as pd
import numpy as np
import nltk
import re
from textblob import TextBlob
from gensim import corpora, models, similarities

In [171]:
dados = pd.read_csv('../lab_01/data/estadao_noticias_eleicao.csv')
dados = dados.replace(np.nan, '', regex=True)

In [172]:
dados.head()

,timestamp,titulo,subTitulo,conteudo,url,idNoticia
0,2014-12-31T00:00:00Z,PT espera 30 mil pessoas em festa na Esplanada,Objetivo é demonstrar apoio popular a Dilma e ...,BRASÍLIA - Após o desgaste provocado com o lan...,"http://politica.estadao.com.br/noticias/geral,...",1
1,2014-12-31T00:00:00Z,Alckmin toma posse de olho no Planalto,Governador reeleito tenta amarrar tucanos paul...,"Reeleito em outubro, o governador tucano Geral...","http://politica.estadao.com.br/noticias/geral,...",2
2,2014-12-31T00:00:00Z,Seis obstáculos e desafios do segundo mandato ...,"Em meio a escândalo de corrupção, presidente t...",1. Rearranjo das contas A nova equipe econôm...,"http://politica.estadao.com.br/noticias/geral,...",3
3,2014-12-31T00:00:00Z,,Veja as principais fotos do dia e dos eventos ...,,"http://fotos.estadao.com.br/fotos/politica,dil...",4
4,2014-12-31T00:00:00Z,,Veja as principais fotos do dia e dos eventos ...,,"http://fotos.estadao.com.br/fotos/politica,dil...",5


In [173]:
print("Os dados tem %d observações e %d colunas" % dados.shape)

Os dados tem 8716 observações e 6 colunas


In [174]:
dados["text"] = dados["titulo"] + " " + dados["subTitulo"] + " " + dados["conteudo"]

In [175]:
dados.head()

,timestamp,titulo,subTitulo,conteudo,url,idNoticia,text
0,2014-12-31T00:00:00Z,PT espera 30 mil pessoas em festa na Esplanada,Objetivo é demonstrar apoio popular a Dilma e ...,BRASÍLIA - Após o desgaste provocado com o lan...,"http://politica.estadao.com.br/noticias/geral,...",1,PT espera 30 mil pessoas em festa na Esplanada...
1,2014-12-31T00:00:00Z,Alckmin toma posse de olho no Planalto,Governador reeleito tenta amarrar tucanos paul...,"Reeleito em outubro, o governador tucano Geral...","http://politica.estadao.com.br/noticias/geral,...",2,Alckmin toma posse de olho no Planalto Governa...
2,2014-12-31T00:00:00Z,Seis obstáculos e desafios do segundo mandato ...,"Em meio a escândalo de corrupção, presidente t...",1. Rearranjo das contas A nova equipe econôm...,"http://politica.estadao.com.br/noticias/geral,...",3,Seis obstáculos e desafios do segundo mandato ...
3,2014-12-31T00:00:00Z,,Veja as principais fotos do dia e dos eventos ...,,"http://fotos.estadao.com.br/fotos/politica,dil...",4,Veja as principais fotos do dia e dos eventos...
4,2014-12-31T00:00:00Z,,Veja as principais fotos do dia e dos eventos ...,,"http://fotos.estadao.com.br/fotos/politica,dil...",5,Veja as principais fotos do dia e dos eventos...


In [176]:
def remove_stopwords (text):
    stopwords = set(nltk.corpus.stopwords.words('portuguese'))
    words = [i for i in text.split() if not i in stopwords]
    return (" ".join(words))

def keep_letters_only(text):
    return re.sub(r'[0-9]+', " ", text)

def standardize_text(text):
    text = text.lower()
    text = remove_stopwords(text)
    text = keep_letters_only(text)
    return text

dados.text = dados.text.apply(standardize_text)

In [177]:
dados.head()["text"]

0    pt espera   mil pessoas festa esplanada objeti...
1    alckmin toma posse olho planalto governador re...
2    seis obstáculos desafios segundo mandato dilma...
3    veja principais fotos dia eventos especiais br...
4    veja principais fotos dia eventos especiais br...
Name: text, dtype: object

In [178]:
def get_palavras(data):
    palavras = []
    for index, row in data.iterrows():
        row["text"] = row["text"].lower()
        palavras_doc = TextBlob(row["text"]).words
        palavras.append(palavras_doc)
    return palavras

In [179]:
texts = get_palavras(dados)

In [180]:
# remove words that appear only once
# from collections import defaultdict
# frequency = defaultdict(int)
# for text in texts:
#     for token in text:
#         frequency[token] += 1
# texts = [[token for token in text if frequency[token] > 1] for text in texts]

In [181]:
dictionary = corpora.Dictionary(texts)
print(dictionary)

Dictionary(59027 unique tokens: ['a', 'abrigo', 'adversários', 'advogado', 'afirma']...)


In [182]:
corpus = [dictionary.doc2bow(text) for text in texts]

In [183]:
lsi = models.LsiModel(corpus, id2word=dictionary, num_topics=2)

In [184]:
index = similarities.MatrixSimilarity(lsi[corpus]) # transform corpus to LSI space and index it

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [204]:
doc = "Alckmin"
vec_bow = dictionary.doc2bow(doc.lower().split())
vec_lsi = lsi[vec_bow] # convert the query to LSI space
print(vec_lsi)

[(0, 0.0217562991126898), (1, 0.009176232764250381)]


In [205]:
sims = index[vec_lsi]
sims = sorted(enumerate(sims), key=lambda item: -item[1]) # perform a similarity query against the corpus
print(sims[0:3])

[(296, 1.0), (1122, 1.0), (2050, 1.0)]


In [206]:
for i in range(0,3):
    print(dictionary[sims[i][0]])

vazio
consumidores
royalties


## Realizando Buscas

In [207]:
def calcula_indices_invertidos(data):
    indices = {}
    for index, row in data.iterrows():
        palavras = TextBlob(row["text"]).words
        
        for palavra in palavras:
            palavra_low = palavra.lower()
            if(palavra_low in indices):
                indices[palavra_low].add(row["idNoticia"])
            else:
                indices[palavra_low] = {row["idNoticia"]}
                
    return(indices)

indices_invertidos = calcula_indices_invertidos(dados)

In [222]:
def busca_um_termo(termo):
    try:
        return(indices_invertidos[termo])
    except KeyError:
        return(set())

def busca_or(termo1, termo2, termo3, termo4):
    try:
        busca1 = indices_invertidos[termo1]
    except KeyError:
        busca1 = set()
        
    try:
        busca2 = indices_invertidos[termo2]
    except KeyError:
        busca2 = set()
        
    try:
        busca3 = indices_invertidos[termo3]
    except KeyError:
        busca3 = set()
    
    try:
        busca4 = indices_invertidos[termo4]
    except KeyError:
        busca4 = set()
        
    return(busca1 | busca2 | busca3 | busca4)


In [215]:
def busca(*args):
    if (len(args) != 4 and len(args) != 1):
        return("Número inválido de argumentos")
    elif(len(args) == 1):
        termo = args[0].lower()
        return(busca_um_termo(termo))
    elif(len(args) == 4):
        termo1 = args[0].lower()
        termo2 = args[1].lower()
        termo3 = args[2].lower()
        termo4 = args[3].lower()
        return(busca_or(termo1, termo2, termo3, termo4))

In [223]:
print(len(busca("Alckmin", "vazio", "consumidores", "royalties")))

764
